In [25]:
import pandas as pd
import cryptpandas as crp


In [ ]:
from slack_sdk import WebClient
import re
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2 import service_account
import io
import cryptpandas as crp

slack_token = ""
channel_id = "C080P6M4DKL"
expected_user_email = "------@gmail.com"

# Google Drive API credentials
SERVICE_ACCOUNT_FILE = 'direct-volt-441912-r9-6b6d82e81e93.json'  # Replace with the path to your JSON key file
SCOPES = ['https://www.googleapis.com/auth/drive']
FOLDER_ID = '1ElVOO_4Plr24xEOmdqsINmIRM_y4M3_n'


# Initialize the Slack client
client = WebClient(token=slack_token)


def fetch_recent_messages(channel_id, limit=30):
    """
    Fetch recent messages from a Slack channel.
    """
    try:
        response = client.conversations_history(channel=channel_id, limit=limit)
        messages = response.get('messages', [])
        return messages
    except Exception as e:
        print(f"Error fetching messages: {e}")
        return []


def get_user_email(user_id):
    """
    Fetch the email of a user based on their Slack user ID.
    """
    try:
        response = client.users_info(user=user_id)
        email = response['user']['profile'].get('email', '')
        return email
    except Exception as e:
        print(f"Error fetching user info: {e}")
        return None


def extract_data_file_and_passcode(messages):
    """
    Extract the data file name and passcode from the messages.
    """
    for message in messages:
        text = message.get('text', '')
        user_id = message.get('user', '')
        # Verify the user's email
        user_email = get_user_email(user_id)
        if user_email == expected_user_email:
            match = re.search(
                r"Data has just been released '(.+?)' the passcode is '(.+?)'\.",
                text
            )
            if match:
                file_name = match.group(1)
                passcode = match.group(2)
                return file_name, passcode
    return None, None


def authenticate_google_drive():
    """
    Authenticate and initialize the Google Drive API client.
    """
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    service = build('drive', 'v3', credentials=credentials)
    return service


def find_and_download_file(service, file_name):
    """
    Find a file by name in the specified folder and download it.
    """
    query = f"'{FOLDER_ID}' in parents and name='{file_name}'"
    results = service.files().list(q=query, spaces='drive').execute()
    files = results.get('files', [])

    if not files:
        print(f"File '{file_name}' not found in the folder.")
        return False

    file_id = files[0]['id']
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(file_name, 'wb')
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        print(f"Download {int(status.progress() * 100)}% complete.")
    
    print(f"File '{file_name}' downloaded successfully.")
    return True


def decrypt_file(file_name, passcode):
    """
    Decrypt the downloaded file using the provided passcode.
    """
    try:
        decrypted_df = crp.read_encrypted(path=file_name, password=passcode)
        decrypted_df.to_csv('latest_data.csv', index=False)
        print("Decrypted DataFrame:")
        print(decrypted_df)
    except Exception as e:
        print(f"Error decrypting file: {e}")


if __name__ == "__main__":
    # Step 1: Fetch recent messages from the Slack channel
    print("Fetching recent messages...")
    messages = fetch_recent_messages(channel_id)

    if not messages:
        print("No messages found or error in fetching messages.")
        exit()

    # Step 2: Extract file name and passcode
    file_name, passcode = extract_data_file_and_passcode(messages)

    if not (file_name and passcode):
        print("No matching message found or sender did not match.")
        exit()

    print(f"Extracted Data:")
    print(f"  File Name: {file_name}")
    print(f"  Passcode: {passcode}")

    # Step 3: Authenticate Google Drive
    print("Authenticating with Google Drive...")
    service = authenticate_google_drive()

    # Step 4: Download the file from Google Drive
    print(f"Downloading file: {file_name}")
    if not find_and_download_file(service, file_name):
        print("File download failed.")
        exit()

    # Step 5: Decrypt the file
    print(f"Decrypting file: {file_name}")
    decrypt_file(file_name, passcode)


Fetching recent messages...
Extracted Data:
  File Name: release_8411.crypt
  Passcode: txpr5wCyFGPpKoqK
Authenticating with Google Drive...
Download 100% complete.
File 'release_8411.crypt' downloaded successfully.
Decrypting file: release_8411.crypt
Decrypted DataFrame:
       strat_0   strat_1   strat_2   strat_3    strat_4    strat_5   strat_6  \
0          NaN       NaN       NaN       NaN        NaN        NaN       NaN   
1    -1.045104       NaN       NaN       NaN        NaN  10.139780  1.481773   
2    -0.229267       NaN       NaN       NaN        NaN  10.109780  0.731916   
3    -0.332475       NaN -0.056052       NaN        NaN   9.961956 -0.557418   
4    -0.827446       NaN -0.349031       NaN        NaN   9.964834 -1.177777   
...        ...       ...       ...       ...        ...        ...       ...   
8407  0.419779 -0.521346  1.176597  1.955253   9.829158  10.269232 -0.463687   
8408 -1.643896 -0.139856 -0.241196  0.537176  10.345547   9.700768 -0.523140   
8409 -1

In [27]:
decrypted_df = crp.read_encrypted(path=file_name, password=passcode)

#   File Name: release_8027.crypt
#   Passcode: xwDXIhUaJe9V4bVR
#   File Name: release_5467.crypt

#   File Name: release_5595.crypt
#   Passcode: bsCATJi4sHpcSZFZ
# #
#   Passcode: CfEJCI8NjykYKV73
# done :release_3547: oUFtGMsMEEyPCCP6, release_3611 : GMJVDf4WWzsV1hfL, release_3675: PSI9bPh4aM3iQMuE
# release_3739: 1vA9LaAZDTEKPePs,release_3803 : 0n74wuaJ2wm8A4qC, release_3867 : mXTi0PZ5oL731Zqx
# release_3931: hjhMuDFZTCJEcI6q
# release_3995 : uZwgENGlQ4m4nSz6
# release_4059 : HkpYXpKituGernrk
# release_4123 : WM5xrwsJiBCo4Unp
# release_4187 : InhVD4qy1Vmbpl5c

# release_4251 : VRiYLce0BKVSdrft
# release_4315 : UR7git7mIlV7HdyY
# release_4379 : scZ5oIa3vp1uzE4S

# release_4571 ! WPcFk8FofuBmfUzO
# release_4635 : hpuTAsG3v5av6J0D
# release_4699 : iqKVo38SFhsSz9qX

# release_4891 : 9YAePHMaPzZKmsmv

#   File Name: release_4955.crypt
#   Passcode: sL7KZHNduMf2Cy5D

#   File Name: release_5275.crypt
#   Passcode: ljo2F8g5X6TWJXmX

#  File Name: release_5403.crypt
#   Passcode: F8CvtiKofpaT7IeO
# File Name: release_5531.crypt
#   Passcode: 9xzhWOAuImRy1cQB
# df_filled = pd.read_csv('data.csv')

In [28]:
df_filled = decrypted_df.ffill().bfill().fillna(decrypted_df.mean())
import pandas as pd
import numpy as np

# Remplacer les valeurs infinies par NaN
df_filled.replace([np.inf, -np.inf], np.nan, inplace=True)

# Vérifier combien de valeurs NaN sont présentes
print("Nombre total de valeurs NaN après remplacement des infinis :", df_filled.isna().sum().sum())

# Option 1 : Supprimer les lignes contenant des NaN
# returns.dropna(inplace=True)

# Option 2 : Remplacer les NaN par une autre valeur, comme la moyenne de chaque colonne
df_filled.fillna(df_filled.mean(), inplace=True)

# Vérifier s'il reste des NaN après le traitement
print("Nombre total de valeurs NaN après traitement :", df_filled.isna().sum().sum())

returns = df_filled

Nombre total de valeurs NaN après remplacement des infinis : 3
Nombre total de valeurs NaN après traitement : 0


In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from scipy.optimize import minimize
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Assume 'returns' is your DataFrame with historical returns of 49 strategies
# For example: returns = pd.read_csv('your_returns_data.csv', index_col='Date', parse_dates=True)

# Since the actual data is not provided, the following code assumes 'returns' is already loaded
# If 'df_filled' is your DataFrame, ensure it is properly loaded
# For the purpose of this code, we'll proceed as if 'returns' is your data

# Data Preprocessing
# Replace infinite values with NaN and drop columns with NaNs
returns = returns.replace([np.inf, -np.inf], np.nan)
print("Nombre total de valeurs NaN après remplacement des infinis :", returns.isna().sum().sum())
returns = returns.dropna(axis=1)
print("Nombre total de valeurs NaN après traitement :", returns.isna().sum().sum())
print("Returns DataFrame shape after preprocessing:", returns.shape)

# Scale the returns data
scaler = StandardScaler()
returns_scaled = pd.DataFrame(scaler.fit_transform(returns), columns=returns.columns, index=returns.index)
print("Returns DataFrame after scaling:")
print(returns_scaled.head())

# Function to create lagged features
def create_lagged_features(data, lag=1):
    df = data.copy()
    column_name = df.columns[0]
    for i in range(1, lag+1):
        df[f'Lag_{i}'] = df[column_name].shift(i)
    return df.dropna()

lag = 5  # Number of lagged days to use as features
predicted_returns = pd.DataFrame(index=returns_scaled.index, columns=returns_scaled.columns)

for asset in returns_scaled.columns:
    print(f"Training model for {asset}...")
    data = returns_scaled[[asset]]
    data_lagged = create_lagged_features(data, lag)
    X = data_lagged.drop(asset, axis=1)
    y = data_lagged[asset]

    # Time series split
    tscv = TimeSeriesSplit(n_splits=5)
    predictions = pd.Series(index=y.index)

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]

        # XGBoost regressor
        model = XGBRegressor(objective='reg:squarederror', n_estimators=100)
        model.fit(X_train, y_train)

        # Predict
        y_pred = model.predict(X_test)
        predictions.iloc[test_index] = y_pred

    predicted_returns[asset] = predictions

# Drop any rows with NaN predictions
predicted_returns = predicted_returns.dropna()

# **Data Cleaning: Clipping Extreme Values in Predicted Returns**
# Clip predicted returns to a reasonable range (e.g., between -3 and 3 standard deviations)
predicted_returns = predicted_returns.clip(lower=-3, upper=3)

# Re-scale predicted returns back to original scale
predicted_returns_unscaled = pd.DataFrame(
    scaler.inverse_transform(predicted_returns),
    columns=predicted_returns.columns,
    index=predicted_returns.index
)

# Calculate expected returns from the predictions
expected_returns = predicted_returns_unscaled.mean()
print("\nExpected Returns per Asset:")
print(expected_returns)

# Calculate the covariance matrix from historical data (scaled returns)
cov_matrix = returns_scaled.loc[predicted_returns.index].cov()

# **Optimization**

def portfolio_performance(weights, expected_returns, cov_matrix):
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_volatility

def neg_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate=0.01):
    p_return, p_volatility = portfolio_performance(weights, expected_returns, cov_matrix)
    sharpe_ratio = (p_return - risk_free_rate) / p_volatility
    return -sharpe_ratio  # Negative because we minimize

# Constraints: Sum of weights = 1, weights between 0% and 10%
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
bounds = tuple((0, 0.1) for _ in range(len(expected_returns)))

# Initial guess
num_assets = len(expected_returns)
initial_weights = num_assets * [1. / num_assets,]

# Optimization
options = {'disp': False}
result = minimize(neg_sharpe_ratio, initial_weights, args=(expected_returns, cov_matrix),
                  method='SLSQP', bounds=bounds, constraints=constraints, options=options)

optimal_weights = result.x
p_return, p_volatility = portfolio_performance(optimal_weights, expected_returns, cov_matrix)
sharpe_ratio = (p_return - 0.01) / p_volatility

# Display results
print("\nOptimal portfolio allocation:")
for i, weight in enumerate(optimal_weights):
    if weight > 0:
        print(f"  {returns.columns[i]}: {weight:.16f}")
print(f"Expected portfolio return: {p_return:.2%}")
print(f"Portfolio volatility: {p_volatility:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")


Nombre total de valeurs NaN après remplacement des infinis : 0
Nombre total de valeurs NaN après traitement : 0
Returns DataFrame shape after preprocessing: (8412, 98)
Returns DataFrame after scaling:
    strat_0   strat_1   strat_2   strat_3  strat_4   strat_5   strat_6  \
0 -0.937183  0.045734 -0.053409  0.147609 -0.96883  0.116622  1.433698   
1 -0.937183  0.045734 -0.053409  0.147609 -0.96883  0.116622  1.433698   
2 -0.210877  0.045734 -0.053409  0.147609 -0.96883  0.088900  0.717783   
3 -0.302759  0.045734 -0.053409  0.147609 -0.96883 -0.047701 -0.513192   
4 -0.743411  0.045734 -0.330009  0.147609 -0.96883 -0.045042 -1.105471   

    strat_7   strat_8   strat_9  ...  strat_88  strat_89  strat_90  strat_91  \
0 -0.271115  1.116291 -0.010904  ... -0.550829  0.653601 -1.065846 -0.699212   
1 -0.271115  1.116291 -0.010904  ... -0.550829  0.653601 -1.065846 -0.699212   
2 -0.271115  1.510500 -0.010904  ... -0.315475  0.653601 -1.065846 -0.699212   
3 -0.271115  0.016834 -0.010904  .

In [ ]:
def create_submission_dict(optimal_weights):
    # Map weights to asset names
    pos_dict = {}
    for i, weight in enumerate(optimal_weights):
        asset_name = returns.columns[i]
        pos_dict[asset_name] = weight

    # Submission checker functions
    def get_positions(pos_dict):
        pos = pd.Series(pos_dict)
        pos = pos.replace([np.inf, -np.inf], np.nan)
        pos = pos.dropna()
        pos = pos / pos.abs().sum()
        pos = pos.clip(-0.1, 0.1)
        if pos.abs().max() / pos.abs().sum() > 0.1:
            raise ValueError(f"Portfolio too concentrated {pos.abs().max()=} / {pos.abs().sum()=}")
        return pos

    def get_submission_dict(
        pos_dict,
        your_team_name: str = "moonwalkers",
        your_team_passcode: str = "osihamedmarkmoonwalkers",
    ):
        positions = get_positions(pos_dict)
        # Format the weights to 20 decimal places as strings
        positions = positions.apply(lambda x: f"{x:.20f}")
        return {
            **positions.to_dict(),
            **{
                "team_name": your_team_name,
                "passcode": your_team_passcode,
            },
        }

    # Create the submission dictionary
    submission_dict = get_submission_dict(
        pos_dict,
        your_team_name="moonwalkers",
        your_team_passcode="osihamedmarkmoonwalkers",
    )

    return submission_dict

submission_answer = create_submission_dict(optimal_weights)
print(submission_answer)

submission_string = str(submission_answer)
print(submission_string)



{'strat_0': '0.00000000000000000000', 'strat_1': '0.00397987945309396119', 'strat_2': '0.00000000009214547768', 'strat_3': '0.00000000000000000000', 'strat_4': '0.09999999959219466050', 'strat_5': '0.09999999959219466050', 'strat_6': '0.00000000000000000000', 'strat_7': '0.00000000002941315324', 'strat_8': '0.00000000003147299615', 'strat_9': '0.00404781565720488314', 'strat_10': '0.05324280231212252107', 'strat_11': '0.00000000000000000000', 'strat_12': '0.00677235793969342779', 'strat_13': '0.05524317356039924742', 'strat_14': '0.00623236969751760853', 'strat_15': '0.00000000000000000000', 'strat_16': '0.00000000000000000000', 'strat_17': '0.00000000000000000000', 'strat_18': '0.03324221790581163849', 'strat_19': '0.00000000000218608775', 'strat_20': '0.09999999959219466050', 'strat_21': '0.00000000000000000000', 'strat_22': '0.04565746553096412808', 'strat_23': '0.00000000000000000000', 'strat_24': '0.00000000000000000000', 'strat_25': '0.00000000000000000000', 'strat_26': '0.000000

In [31]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# import time
# import os


# # Define the form URL
# form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSeUYMkI5ce18RL2aF5C8I7mPxF7haH23VEVz7PQrvz0Do0NrQ/viewform'


# options = webdriver.ChromeOptions()
# options.add_argument(r"--user-data-dir=C:\Users\theha\AppData\Local\Google\Chrome\User Data") #e.g. C:\Users\You\AppData\Local\Google\Chrome\User Data
# options.add_argument(r'--profile-directory=Profile 1') #e.g. Profile 3


# driver = webdriver.Chrome(options=options)

# try:
#     # Step 1: Navigate to the Google Form
#     driver.get(form_url)
#     time.sleep(3)  # Wait for the page to load

#     # Step 2: Press the checkbox button
#     checkbox = driver.find_element(By.XPATH, '//div[@id="i5"]')
#     checkbox.click()
#     time.sleep(1)  # Allow time for the checkbox to register

#     # Step 3: Fill out the textarea field with your answer
#     # textarea = driver.find_element(By.XPATH, '//textarea[@class="KHxj8b tL9Q4c"][@aria-label="Your answer"]')

#     textarea_container = driver.find_element(By.XPATH, '//div[@class="edhGSc zKHdkd kRy7qc RdH0ib yqQS1"]')
#     textarea = textarea_container.find_element(By.XPATH, './/textarea[@aria-label="Your answer"]')
#     textarea.send_keys(submission_string)  # Replace with your answer
#     time.sleep(2)  # Allow time for input

#     # Step 4: Press the Submit button
#     submit_button = driver.find_element(By.XPATH, '//span[@class="NPEfkd RveJvd snByac"][text()="Submit"]')
#     submit_button.click()
#     time.sleep(3)  # Wait for the submission to process

#     print("Form submitted successfully.", submission_string)

# except Exception as e:
#     print(f"An error occurred: {e}")

# finally:
#     driver.quit()


In [32]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# from scipy.optimize import minimize

# # Supposons que 'returns' soit un DataFrame avec les rendements des stratégies
# # Exemple : returns = pd.DataFrame({...})

# # Suppression des NaNs et des valeurs infinies
# returns = returns.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
# print("Returns DataFrame shape after preprocessing:", returns.shape)

# # Check if returns DataFrame is empty
# if returns.empty:
#     raise ValueError("The 'returns' DataFrame is empty after preprocessing. Please check your data.")

# # **Standardize the data for PCA**
# scaler = StandardScaler()
# returns_scaled = pd.DataFrame(scaler.fit_transform(returns), columns=returns.columns)
# print("Returns DataFrame after scaling:")
# print(returns_scaled.head())

# # **Calculate mean returns and covariance matrix from original data**
# mean_returns = returns.mean()
# cov_matrix = returns.cov()

# # **Perform PCA on the scaled data**
# pca = PCA()
# pca.fit(returns_scaled)

# # Verify PCA components
# print("Explained Variance Ratio:", pca.explained_variance_ratio_)
# print("Total Explained Variance:", np.sum(pca.explained_variance_ratio_))

# # Keep components that explain 80% of the variance
# cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
# n_components = np.argmax(cumulative_variance >= 0.8) + 1
# print(f"Number of components to reach 80% variance: {n_components}")

# # Obtain the top principal components
# top_components = pca.components_[:n_components]
# print("Top Components Shape:", top_components.shape)

# # Calculate the contribution of each strategy to the principal components
# strategy_contributions = np.sum(np.abs(top_components), axis=0)
# print("Strategy Contributions:", strategy_contributions)

# # Create a DataFrame to list strategies and their contributions
# strategy_contributions_df = pd.DataFrame({
#     'Strategy': returns.columns,
#     'Contribution': strategy_contributions
# })

# # Verify if total_contribution is zero
# total_contribution = strategy_contributions_df['Contribution'].sum()
# print("Total Contribution:", total_contribution)

# if total_contribution == 0:
#     raise ValueError("Total contribution is zero. Cannot compute cumulative percentage contribution. Please check your data.")
# else:
#     # Sort strategies by descending contribution
#     strategy_contributions_df = strategy_contributions_df.sort_values(by='Contribution', ascending=False)

#     # Calculate cumulative contribution
#     strategy_contributions_df['CumulativeContribution'] = strategy_contributions_df['Contribution'].cumsum()
#     strategy_contributions_df['CumulativeContributionPct'] = strategy_contributions_df['CumulativeContribution'] / total_contribution

#     # Display the contributions DataFrame
#     print("\nStrategy Contributions DataFrame:")
#     print(strategy_contributions_df)

#     # Select strategies that cumulatively contribute up to 80%
#     selected_strategies = strategy_contributions_df[strategy_contributions_df['CumulativeContributionPct'] <= 0.8]['Strategy']

#     # If no strategies are selected, adjust the threshold or selection method
#     if selected_strategies.empty:
#         # Option: Select the top N strategies
#         N = 10  # For example, select top 10 strategies
#         selected_strategies = strategy_contributions_df['Strategy'].head(N)
#         print(f"\nNo strategies met the 80% cumulative contribution threshold. Selecting top {N} strategies instead.")

#     # **Subset original data for selected strategies**
#     returns_selected = returns[selected_strategies]
#     mean_returns_selected = mean_returns[selected_strategies]
#     cov_matrix_selected = cov_matrix.loc[selected_strategies, selected_strategies]

#     # Portfolio optimization parameters
#     num_assets = len(selected_strategies)
#     initial_weights = [1 / num_assets] * num_assets
#     bounds = tuple((0, 0.1) for _ in range(num_assets))
#     risk_free_rate = 0.01

#     # Constraints for weights summing to 1
#     constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

#     # Function to calculate portfolio performance
#     def portfolio_performance(weights, mean_returns, cov_matrix):
#         port_return = np.dot(weights, mean_returns)
#         port_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
#         return port_return, port_std

#     # Objective function to minimize volatility
#     def minimize_volatility(weights):
#         return portfolio_performance(weights, mean_returns_selected, cov_matrix_selected)[1]

#     # Perform optimization
#     min_vol_allocation = minimize(minimize_volatility, initial_weights,
#                                   method='SLSQP', bounds=bounds, constraints=constraints)

#     # Check optimization success
#     if not min_vol_allocation.success:
#         print("Optimization failed:", min_vol_allocation.message)
#     else:
#         # Extract optimal weights
#         optimal_weights = min_vol_allocation.x

#         # Calculate portfolio performance
#         opt_return, opt_volatility = portfolio_performance(optimal_weights, mean_returns_selected, cov_matrix_selected)

#         # Calculate Sharpe Ratio
#         sharpe_ratio = (opt_return - risk_free_rate) / opt_volatility

#         # Display results
#         print(f"\nAllocation optimale avec PCA et stratégies priorisées :")
#         for i, weight in enumerate(optimal_weights):
#             if abs(weight) > 0:  # Display only non-zero positions
#                 print(f"  {selected_strategies.iloc[i]}: {weight:.2%}")
#         print(f"Rendement attendu: {opt_return:.2%}")
#         print(f"Volatilité: {opt_volatility:.2%}")
#         print(f"Ratio de Sharpe: {sharpe_ratio:.2f}")
#         print("-" * 50)

#         # Convert allocations to DataFrame
#         allocations_df_pca = pd.DataFrame([optimal_weights], columns=returns_selected.columns)

#         # Save to CSV
#         allocations_df_pca.to_csv('optimal_allocations_pca.csv', index=False)

#         # Display allocations DataFrame
#         print("\nDataFrame des allocations optimales basées sur PCA :")
#         print(allocations_df_pca)

#         # Optional: Visualization
#         plt.figure(figsize=(10, 6))
#         plt.barh(returns_selected.columns, optimal_weights, color='skyblue')
#         plt.xlabel('Poids des actifs')
#         plt.title('Allocation optimale basée sur PCA et stratégies priorisées')
#         plt.show()
